In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../Dados/CSV/point5000.csv')

In [3]:
df.head()

,id,epac,USO_2015,B1,B2,B3,B4,B5,B6,B7,B1.1,B2.1,B3.1,B4.1,B5.1,B6.1,B7.1
0,887,a,ROCHA,8437.818438,7719.032383,7173.540135,6628.093034,11887.028173,10047.598934,7666.984792,0.427021,0.391512,0.364547,0.337578,0.597521,0.325896,0.251033
1,1174,a,ROCHA,8465.084335,7742.233643,7332.444501,6595.879132,15583.545194,10903.904403,7845.405297,0.428366,0.392656,0.372402,0.335987,0.780239,0.352816,0.256646
2,4272,a,ROCHA,8101.581027,7299.567316,6300.519469,5750.398151,5947.106534,5215.316921,5081.238643,0.410403,0.370778,0.321395,0.294193,0.303915,0.173958,0.169733
3,4884,a,ROCHA,8436.934048,7680.410630,6868.116697,6368.261542,9259.926548,8044.223838,6921.642563,0.426977,0.389603,0.349450,0.324734,0.467661,0.262901,0.227598
4,4915,a,ROCHA,8480.928720,7751.628078,7294.537935,6567.613085,16023.725229,10505.149220,7272.544194,0.429155,0.393122,0.370530,0.334592,0.801994,0.340278,0.238630


# Preparando a variável x e y

In [4]:
# y
y = df.iloc[:,2]

In [6]:
from sklearn.preprocessing import OrdinalEncoder,CategoricalEncoder, LabelEncoder,OneHotEncoder, LabelBinarizer

In [247]:
# Cat de forma binária 
le = LabelBinarizer()

In [248]:
y= le.fit_transform(y)

In [249]:
yL=pd.DataFrame(y,columns=le.classes_).head()

In [250]:
yL

,AGRICULTURA,AGUA,AUMIDAS,FLORESTA,MANGUE,PASTAGEM,ROCHA,SOLO,URBANO
0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0


In [251]:
le.classes_

array(['AGRICULTURA', 'AGUA', 'AUMIDAS', 'FLORESTA', 'MANGUE', 'PASTAGEM',
       'ROCHA', 'SOLO', 'URBANO'], dtype='<U11')

In [252]:
y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
from sklearn.externals import joblib

In [17]:
joblib.dump(le,'../modelos/inverse.pk1')

['../modelos/inverse.pk1']

In [19]:
# x
x =df.iloc[:,6:9]

In [18]:
df.iloc[:,6:9].head()

,B4,B5,B6
0,6628.093034,11887.028173,10047.598934
1,6595.879132,15583.545194,10903.904403
2,5750.398151,5947.106534,5215.316921
3,6368.261542,9259.926548,8044.223838
4,6567.613085,16023.725229,10505.149220


In [20]:
#padronizando 

from sklearn.preprocessing import StandardScaler

In [21]:
Stand = StandardScaler()
x = Stand.fit(x).transform(x)

In [22]:
# separando 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

# Testando algoritmos

In [23]:
# Algoritmos
# importando bibliotecas
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier  # permite utilizar algoritmos com SVC em Labelbinariser

In [24]:
# plota grafico de dispersao e linha de tendencia para o modelo
def plot_show(X_test,y_test,previsao,title,name):
    #plt.figure(figsize=(5,5))
    z= np.polyfit(y_test.flatten(),previsao.flatten(),1)
    p=np.poly1d(z)
    plt.scatter(y_test,previsao,color='k',marker='+')
    plt.title(title)
    plt.plot(y_test,p(y_test),'r--')
    plt.show()
    print('RMSE for ',name,' is ',y_mse[name])
    print('R**2 ',name,' is ',y_r2[name])

In [25]:
# plota grafico de dispersao e linha de tendencia para o modelo
def plot_l(y_test,previsao):
    #plt.figure(figsize=(5,5))
    z= np.polyfit(y_test.flatten(),previsao.flatten(),1)
    p=np.poly1d(z)
    return p

In [27]:
# cria dicionario com os algorimos utilizados 
ESTIMATORS = {    
    "K-nn": KNeighborsClassifier(),                        
    "RandomForestRegressor": RandomForestClassifier(max_depth=50, random_state=2),
    "Decision Tree Regressor":DecisionTreeClassifier(max_depth=5),
    "SVC":OneVsRestClassifier(estimator=SVC()),
    "NeuralNetwork": MLPClassifier(solver='adam', random_state=1,
                                  hidden_layer_sizes=(4,4,4,2)),
    "MultiO/P GBR" :OneVsRestClassifier(estimator=GradientBoostingClassifier(n_estimators=5)),
    "MultiO/P AdaB" :OneVsRestClassifier(estimator=AdaBoostClassifier(n_estimators=5))
}


In [28]:
estimator = OneVsRestClassifier(estimator=GradientBoostingClassifier(n_estimators=5))


In [29]:
estimator.fit(X_train,y_train)

OneVsRestClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          n_jobs=None)

In [30]:
estimator.score(X_test,y_test)

0.5600728155339806

# Ensaios 

In [123]:
# x
x =df.iloc[:,11:14]

In [121]:
df.iloc[:,11:14].head()

,B2.1,B3.1,B4.1
0,0.391512,0.364547,0.337578
1,0.392656,0.372402,0.335987
2,0.370778,0.321395,0.294193
3,0.389603,0.349450,0.324734
4,0.393122,0.370530,0.334592


In [122]:
# x
x =df[['B1','B2','B6']]

In [233]:
Stand = StandardScaler()
x = Stand.fit(x).transform(x)

In [124]:
# separando 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

In [125]:
# cria dicionario com os algorimos utilizados 
ESTIMATORS = {    
    "K-nn": KNeighborsClassifier(),                        
    "RandomForestRegressor": RandomForestClassifier(max_depth=50, random_state=2),
    "Decision Tree Regressor":DecisionTreeClassifier(max_depth=5),
    "SVC":OneVsRestClassifier(estimator=SVC()),
    "NeuralNetwork": MLPClassifier(solver='adam', random_state=1,
                                  hidden_layer_sizes=(4,4,4,2)),
    "MultiO/P GBR" :OneVsRestClassifier(estimator=GradientBoostingClassifier(n_estimators=5)),
    "MultiO/P AdaB" :OneVsRestClassifier(estimator=AdaBoostClassifier(n_estimators=5))
}


In [275]:
# aplicando os algortimos
y_test_predict = dict()
y_mse = dict()
y_r2 = dict()
#fig = plt.figure(figsize=(15,10))
i=1
for name, estimator in ESTIMATORS.items():    
    estimator.fit(X_train, y_train)                    # fit() with instantiated object
    y_test_predict[name] = previsao = estimator.predict(X_test)   # Make predictions and save it in dict under key: name
    y_mse[name] = mean_squared_error(y_test, estimator.predict(X_test))
    y_r2[name] = estimator.score(X_test,y_test)
    
    #grafico
#     z= np.polyfit(y_test.flatten(),previsao.flatten(),1)
#     p=np.poly1d(z)
#     ax=fig.add_subplot(2,3,i)
#     ax.scatter(y_test,previsao,color='k',marker='+')
#     ax.set_xlabel('Observado')
#     ax.set_ylabel('Estimado')
#     ax.set_title(name)
#     ax.plot(y_test,p(y_test),'r--')
#     i+=1

# plt.tight_layout()
# #plt.savefig('../../../Qualificacao/MML_evaporacao_INMET.png')
# plt.show()

    

R=pd.DataFrame({'mse':list(y_mse.values()),    # Padronizando o os dois
                 'R2':list(y_r2.values())}, 
                index=list(y_r2.keys()))

/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/

In [276]:
R

,mse,R2
K-nn,0.046049,0.756675
RandomForestRegressor,0.043959,0.739684
Decision Tree Regressor,0.046251,0.787015
SVC,0.041936,0.738471
NeuralNetwork,0.050162,0.669296
MultiO/P GBR,0.052656,0.569782
MultiO/P AdaB,0.043959,0.725728


# Considerando o NDVI e o EVI

In [265]:
df.head()

,id,epac,USO_2015,B1,B2,B3,B4,B5,B6,B7,B1.1,B2.1,B3.1,B4.1,B5.1,B6.1,B7.1
0,887,a,ROCHA,8437.818438,7719.032383,7173.540135,6628.093034,11887.028173,10047.598934,7666.984792,0.427021,0.391512,0.364547,0.337578,0.597521,0.325896,0.251033
1,1174,a,ROCHA,8465.084335,7742.233643,7332.444501,6595.879132,15583.545194,10903.904403,7845.405297,0.428366,0.392656,0.372402,0.335987,0.780239,0.352816,0.256646
2,4272,a,ROCHA,8101.581027,7299.567316,6300.519469,5750.398151,5947.106534,5215.316921,5081.238643,0.410403,0.370778,0.321395,0.294193,0.303915,0.173958,0.169733
3,4884,a,ROCHA,8436.934048,7680.410630,6868.116697,6368.261542,9259.926548,8044.223838,6921.642563,0.426977,0.389603,0.349450,0.324734,0.467661,0.262901,0.227598
4,4915,a,ROCHA,8480.928720,7751.628078,7294.537935,6567.613085,16023.725229,10505.149220,7272.544194,0.429155,0.393122,0.370530,0.334592,0.801994,0.340278,0.238630


In [162]:
df=df.drop(['xcoord','ycoord','xcoord_1','ycoord_1'],axis=1)

In [266]:
def ndvi(df):
    return ((df['B5']-df['B4'])/(df['B5']+df['B4']))

def evi(df):
     return (2.5*(df['B5']-df['B4'])/(df['B5']+6*df['B4']-7.5*df['B2']+1))

In [267]:
df['NDVI']=df.apply(ndvi, axis=1)
df['EVI']=df.apply(evi, axis=1)

In [268]:
#df[['B1','B2','B3','B4','B5','B6']].head()

,B1,B2,B3,B4,B5,B6
0,8437.818438,7719.032383,7173.540135,6628.093034,11887.028173,10047.598934
1,8465.084335,7742.233643,7332.444501,6595.879132,15583.545194,10903.904403
2,8101.581027,7299.567316,6300.519469,5750.398151,5947.106534,5215.316921
3,8436.934048,7680.410630,6868.116697,6368.261542,9259.926548,8044.223838
4,8480.928720,7751.628078,7294.537935,6567.613085,16023.725229,10505.149220


In [8]:
# x
x =df[['B1','B2','B3','B4','B5','B6']]#df.iloc[:,7:10]

In [364]:
Stand = StandardScaler()
x = Stand.fit(x).transform(x)

In [365]:
# separando 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

In [366]:
# cria dicionario com os algorimos utilizados 
ESTIMATORS = {    
    "K-nn": KNeighborsClassifier(),                        
    "RandomForestRegressor": RandomForestClassifier(n_estimators=150,max_depth=3,
                                                    random_state=5),
    "Decision Tree Regressor":DecisionTreeClassifier(max_depth=5),
    "SVC":OneVsRestClassifier(estimator=SVC(kernel='rbf')),
    "NeuralNetwork": MLPClassifier(solver='adam', random_state=1,
                                  hidden_layer_sizes=(4,4,4,2)),
    "MultiO/P GBR" :OneVsRestClassifier(estimator=GradientBoostingClassifier(n_estimators=5)),
    "MultiO/P AdaB" :OneVsRestClassifier(estimator=AdaBoostClassifier(n_estimators=11))
}


In [367]:
### CALCULA O KAPPA E O G

# aplicando os algortimos
y_test_predict = dict()
K = dict()
G = dict()
#fig = plt.figure(figsize=(15,10))
i=1
for name, estimator in ESTIMATORS.items():    
    estimator.fit(X_train, y_train)                    # fit() with instantiated object
    y_test_predict[name] = previsao = estimator.predict(X_test)   # Make predictions and save it in dict under key: name
    y_o = le.inverse_transform(y_test)
    y_e = le.inverse_transform(previsao)
    K[name] = cohen_kappa_score(y_o,y_e)
    G[name] = accuracy_score(y_o,y_e)
    
    #grafico
#     z= np.polyfit(y_test.flatten(),previsao.flatten(),1)
#     p=np.poly1d(z)
#     ax=fig.add_subplot(2,3,i)
#     ax.scatter(y_test,previsao,color='k',marker='+')
#     ax.set_xlabel('Observado')
#     ax.set_ylabel('Estimado')
#     ax.set_title(name)
#     ax.plot(y_test,p(y_test),'r--')
#     i+=1

# plt.tight_layout()
# #plt.savefig('../../../Qualificacao/MML_evaporacao_INMET.png')
# plt.show()

    

Resultado=pd.DataFrame({'G':list(G.values()),    # Padronizando o os dois
                 'K':list(K.values())}, 
                index=list(K.keys()))

/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/pensario/.local/lib/python3.6/site-packages/sklearn/

In [362]:
Resultado

,G,K
K-nn,0.734830,0.607979
RandomForestRegressor,0.726942,0.599434
Decision Tree Regressor,0.759102,0.638340
SVC,0.729369,0.608738
NeuralNetwork,0.626214,0.478730
MultiO/P GBR,0.568568,0.436323
MultiO/P AdaB,0.705097,0.582444


# Ajustes finais em cada algoritmo 

In [368]:
# estimator = MLPClassifier(solver='adam', random_state=1,
#                                   hidden_layer_sizes=(3,3,3,2))

estimator =DecisionTreeClassifier(max_depth=5)

In [369]:
import time
t1=time.clock()

estimator.fit(X_train,y_train)
t2=time.clock()
tf = t2-t1
print(tf)

0.020527999999956137


In [370]:
estimator.score(X_test,y_test)

0.7263349514563107

In [234]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

In [372]:
y_o = le.inverse_transform(y_train)
y_e = le.inverse_transform(estimator.predict(X_train))
cohen_kappa_score(y_o,y_e)

0.6573908282918364

In [373]:
accuracy_score(y_o,y_e)

0.7663775052348191

In [237]:
#matrix de confusao 
from sklearn.metrics import confusion_matrix


In [374]:
confusion_matrix(y_o,y_e)

array([[   8,    0,    0,    2,    0,    4,    0,    0,    3],
       [   1,    3,    0,    0,    0,    0,    0,    0,    0],
       [  19,    0,    8,    3,    2,    0,    0,    0,    2],
       [  93,    0,    0, 1345,    0,   68,    0,    0,   20],
       [  12,    0,    1,    6,   38,    0,    0,    0,    1],
       [ 152,    0,    1,  115,    0,  318,    0,    0,   79],
       [   0,    0,    0,    1,    0,    0,    2,    0,    0],
       [   3,    0,    0,    0,    0,    2,    0,    0,   24],
       [ 106,    0,    0,   26,    0,   35,    0,    0,  840]])

In [375]:
pd.crosstab(y_o,y_e)

col_0,AGRICULTURA,AGUA,AUMIDAS,FLORESTA,MANGUE,PASTAGEM,ROCHA,URBANO
row_0,,,,,,,,
AGRICULTURA,8,0,0,2,0,4,0,3
AGUA,1,3,0,0,0,0,0,0
AUMIDAS,19,0,8,3,2,0,0,2
FLORESTA,93,0,0,1345,0,68,0,20
MANGUE,12,0,1,6,38,0,0,1
PASTAGEM,152,0,1,115,0,318,0,79
ROCHA,0,0,0,1,0,0,2,0
SOLO,3,0,0,0,0,2,0,24
URBANO,106,0,0,26,0,35,0,840


# Exportando os modelos 

In [87]:
from sklearn.externals import joblib

In [226]:
# arvore de decisao, floresta de decisao e knn (2,3,6,NDVI : landsat 8)
joblib.dump(estimator,'../modelos/SVC.pk1', compress=9)

['../modelos/SVC.pk1']

In [14]:
#importando 
clone = joblib.load('../modelos/randon_f.pk1')


In [19]:
estimadores = clone.estimators_[5]

In [30]:
#visualizando arvore
from sklearn.tree import export_graphviz
export_graphviz(estimadores, out_file='dotfile',               
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [28]:
from subprocess import call
from os import system

In [31]:
#call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
system("dot -Tpng D:.dot -o D:/dtree2.png")

32512

In [33]:
from IPython.display import Image
Im(filename = 'D:/dtree2.png')

NameError: name 'Im' is not defined